# Prepare Cut List 

Using the segment ranking from the previous step although with the source transcript, this script creates a cutlist for the final edit.


## Helper function for merge timestamps and segment selections

In [ ]:
def merge_ranges(ranges):
    if not ranges:
        return []

    # Sort ranges by start point
    merged = [ranges[0]]

    for current in ranges[1:]:
        last = merged[-1]
        # If current range starts at or before last ends + 1, merge them
        if current[0] <= last[1] + 1:
            merged[-1] = (last[0], max(last[1], current[1]))
        else:
            merged.append(current)

    return merged

## Helper function for finding the gaps in audio

In [ ]:
# Find the gaps before and after a given timestamp
# To do that, look through the transcript and find the biggest timestamp BEFORE each segments
# and the smallest timestamp AFTER each segment

def find_gap_prev(audio_items, start_time):
    # Go through the end times of all audio items to find any that are before the given start time
    eligible_end_times = [
        float(item.get("end_time")) for item in audio_items if item.get("end_time") and float(item.get("end_time")) < float(start_time)
    ]
    return max(eligible_end_times) if eligible_end_times else None

def find_gap_next(audio_items, end_time):
    # Go through the start times of all audio items to find any that are after the given end time
    eligible_start_times = [
        float(item.get("start_time")) for item in audio_items if item.get("start_time") and float(item.get("start_time")) > float(end_time)
    ]
    return min(eligible_start_times) if eligible_start_times else None

In [ ]:
import json
import csv

VIDEO_ID = 'gYXAulePuXY'

MIN_RANKING = 8 # higher numbers indicate more relevant audio clips

transcription_file = f"content/{VIDEO_ID}-transcript.json"
ranking_file = f"content/{VIDEO_ID}-ranking.json"
output_file = f"content/{VIDEO_ID}-cutlist.json"

# Load the transcription from AWS Transcribe
with open(transcription_file, 'r') as f:
    transcription = json.load(f)

# Load the phrase groupings (from an LLM)
with open(ranking_file, 'r') as f:
    rankings = json.load(f)

# Get the phrases portion of the transcription_file
audio_segments = transcription.get('results', {}).get('audio_segments', [])

# Create a lookup dictionary from source
id_to_segment = {item["id"]: item for item in audio_segments}

# Filter for the segments we'll keep
# (all temporary of course--this part is just an experiment)
selected_segments = [
    (int(ranking.get('ID').split('-')[0]), int(ranking.get('ID').split('-')[-1]))
    for ranking in rankings
    if ranking.get('ranking') >= MIN_RANKING
]

# Join contiguous segments to elimate redundant edits, e.g. [(6, 8), (9, 13)] -> [(6, 13)]
merged_segments = merge_ranges(selected_segments)

# Get the full transcript details for each segment ID 
segment_details = [
    (id_to_segment.get(segment[0]), id_to_segment.get(segment[-1]))
    for segment in merged_segments
]

# Transform segment ID to start and end timestamps
# And find the gaps we have between this segment and other speech.
audio_items = transcription.get('results', {}).get('items', [])
segment_timestamps = [
    {
        'start_time': float(segment[0]['start_time']),
        'end_time': float(segment[1]['end_time']),
        'prev_audio': find_gap_prev(audio_items, segment[0]['start_time']),
        'next_audio': find_gap_next(audio_items, segment[1]['end_time'])
    }
    for segment in segment_details
]

# Taking the start and end times of the previous step, get precise cut points and (max) transition durations
cutlist = [
    {
        **segment,
        'start_cut': segment['start_time'] - (segment['start_time'] - segment['prev_audio']) / 2,
        'start_transition': segment['start_time'] - segment['prev_audio'],
        'end_cut': segment['end_time'] + (segment['next_audio'] - segment['end_time']) / 2,
        'end_transition': segment['next_audio'] - segment['end_time'],
    }
    for segment in segment_timestamps
]

with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(cutlist, json_file, ensure_ascii=False, indent=4)

print(f"Data written to {output_file}")
